In [27]:
import numpy as np 
import pandas as pd 



dataFile='/Users/avinashreddy/Downloads/ml-100k/u.data'
data=pd.read_csv(dataFile,sep="\t",header=None,
                 names=['userId','itemId','rating','timestamp'])



data.head()




movieInfoFile='/Users/avinashreddy/Downloads/ml-100k/u.item'
movieInfo=pd.read_csv(movieInfoFile,sep="|", header=None, index_col=False,
                     names=['itemId','title'], usecols=[0,1])

movieInfo.head()



data=pd.merge(data,movieInfo,left_on='itemId',right_on="itemId")

data.head()



userIds=data.userId # a Pandas series object
userIds2=data[['userId']] # a Pandas DataFrame object




userIds.head()




userIds2.head()




type(userIds)




type(userIds2)



data.loc[0:10,['userId']]











toyStoryUsers=data[data.title=="Toy Story (1995)"]

toyStoryUsers.head()




,userId,itemId,rating,timestamp,title
3397,308,1,4,887736532,Toy Story (1995)
3398,287,1,5,875334088,Toy Story (1995)
3399,148,1,4,877019411,Toy Story (1995)
3400,280,1,4,891700426,Toy Story (1995)
3401,66,1,3,883601324,Toy Story (1995)


In [28]:

data=pd.DataFrame.sort_values(data,['userId','itemId'],ascending=[0,1])


numUsers=max(data.userId)
numMovies=max(data.itemId)


moviesPerUser=data.userId.value_counts()
usersPerMovie=data.title.value_counts()

usersPerMovie



def favoriteMovies(activeUser,N):
    #1. subset the dataframe to have the rows corresponding to the active user
    # 2. sort by the rating in descending order
    # 3. pick the top N rows
    topMovies=pd.DataFrame.sort_values(
        data[data.userId==activeUser],['rating'],ascending=[0])[:N]
    
    return list(topMovies.title)

print favoriteMovies(5,3) # Print the top 3 favorite movies of user 5



userItemRatingMatrix=pd.pivot_table(data, values='rating',
                                    index=['userId'], columns=['itemId'])

userItemRatingMatrix.head()




from scipy.spatial.distance import correlation 
def similarity(user1,user2):
    user1=np.array(user1)-np.nanmean(user1) 
    user2=np.array(user2)-np.nanmean(user2)
    
    commonItemIds=[i for i in range(len(user1)) if user1[i]>0 and user2[i]>0]
    # Gives us movies for which both users have non NaN ratings 
    if len(commonItemIds)==0:
        # If there are no movies in common 
        return 0
    else:
        user1=np.array([user1[i] for i in commonItemIds])
        user2=np.array([user2[i] for i in commonItemIds])
        return correlation(user1,user2)
    



def nearestNeighbourRatings(activeUser,K):
    
    similarityMatrix=pd.DataFrame(index=userItemRatingMatrix.index,
                                  columns=['Similarity'])
   
    for i in userItemRatingMatrix.index:
        similarityMatrix.loc[i]=similarity(userItemRatingMatrix.loc[activeUser],
                                          userItemRatingMatrix.loc[i])
        
    similarityMatrix=pd.DataFrame.sort_values(similarityMatrix,
                                              ['Similarity'],ascending=[0])
    # Sort the similarity matrix in the descending order of similarity 
    nearestNeighbours=similarityMatrix[:K]
    # The above line will give us the K Nearest neighbours 
    
    
    neighbourItemRatings=userItemRatingMatrix.loc[nearestNeighbours.index]
    
    predictItemRating=pd.DataFrame(index=userItemRatingMatrix.columns, columns=['Rating'])
    
    for i in userItemRatingMatrix.columns:
        # for each item 
        predictedRating=np.nanmean(userItemRatingMatrix.loc[activeUser])
        # start with the average rating of the user
        for j in neighbourItemRatings.index:
            # for each neighbour in the neighbour list 
            if userItemRatingMatrix.loc[j,i]>0:
                
                predictedRating += (userItemRatingMatrix.loc[j,i]
                                    -np.nanmean(userItemRatingMatrix.loc[j]))*nearestNeighbours.loc[j,'Similarity']
        
        predictItemRating.loc[i,'Rating']=predictedRating
    return predictItemRating


def topNRecommendations(activeUser,N):
    predictItemRating=nearestNeighbourRatings(activeUser,10)
    # Use the 10 nearest neighbours to find the predicted ratings
    moviesAlreadyWatched=list(userItemRatingMatrix.loc[activeUser]
                              .loc[userItemRatingMatrix.loc[activeUser]>0].index)
    # find the list of items whose ratings which are not NaN
    predictItemRating=predictItemRating.drop(moviesAlreadyWatched)
    topRecommendations=pd.DataFrame.sort_values(predictItemRating,
                                                ['Rating'],ascending=[0])[:N]
    
    topRecommendationTitles=(movieInfo.loc[movieInfo.itemId.isin(topRecommendations.index)])
    return list(topRecommendationTitles.title)
    
    


# In[25]:

# Let's take this for a spin 
activeUser=5
print favoriteMovies(activeUser,5),"\n",topNRecommendations(activeUser,3)





['Men in Black (1997)', 'Blade Runner (1982)', 'Empire Strikes Back, The (1980)']
['Men in Black (1997)', 'Blade Runner (1982)', 'Empire Strikes Back, The (1980)', 'Wrong Trousers, The (1993)', 'Blues Brothers, The (1980)'] 
['Truth About Cats & Dogs, The (1996)', 'Scream (1996)', 'First Wives Club, The (1996)']


In [29]:
def matrixFactorization(R, K, steps=10, gamma=0.001,lamda=0.02):
    #
    N=len(R.index)# Number of users
    M=len(R.columns) # Number of items 
    P=pd.DataFrame(np.random.rand(N,K),index=R.index)
    
    Q=pd.DataFrame(np.random.rand(M,K),index=R.columns)
    
    for step in xrange(steps):
        
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    # For each rating that exists in the training set 
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        
        e=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    #Sum of squares of the errors in the rating
                    e= e + pow(R.loc[i,j]-np.dot(P.loc[i],Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
        print step
    return P,Q


(P,Q)=matrixFactorization(userItemRatingMatrix.iloc[:100,:100],K=2,gamma=0.001,lamda=0.02, steps=100)

activeUser=1
predictItemRating=pd.DataFrame(np.dot(P.loc[activeUser],Q.T),index=Q.index,columns=['Rating'])
topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])[:3]
# We found the ratings of all movies by the active user and then sorted them to find the top 3 movies 
topRecommendationTitles=movieInfo.loc[movieInfo.itemId.isin(topRecommendations.index)]
print list(topRecommendationTitles.title)


0
1
2
3
4
5
6
7
8
9
10
11


KeyboardInterrupt: 

In [30]:


import itertools 


allitems=[]

def support(itemset):
    userList=userItemRatingMatrix.index
    nUsers=len(userList)
    ratingMatrix=userItemRatingMatrix
    for item in itemset:
        ratingMatrix=ratingMatrix.loc[ratingMatrix.loc[:,item]>0]
        #Subset the ratingMatrix to the set of users who have rated this item 
        userList=ratingMatrix.index
    
    return float(len(userList))/float(nUsers)


minsupport=0.3
for item in list(userItemRatingMatrix.columns):
    itemset=[item]
    if support(itemset)>minsupport:
        allitems.append(item)

        






len(allitems)



minconfidence=0.1
assocRules=[]
i=2
for rule in itertools.permutations(allitems,2):
    
    from_item=[rule[0]]
    to_item=rule
    # each rule is a tuple of 2 items 
    confidence=support(to_item)/support(from_item)
    if confidence>minconfidence and support(to_item)>minsupport:
        assocRules.append(rule)


assocRules

# End


[(1, 50),
 (1, 100),
 (1, 117),
 (1, 121),
 (1, 181),
 (7, 50),
 (7, 100),
 (7, 181),
 (50, 1),
 (50, 7),
 (50, 56),
 (50, 69),
 (50, 79),
 (50, 98),
 (50, 100),
 (50, 117),
 (50, 121),
 (50, 127),
 (50, 172),
 (50, 173),
 (50, 174),
 (50, 181),
 (50, 204),
 (50, 210),
 (50, 222),
 (50, 237),
 (50, 258),
 (50, 288),
 (50, 294),
 (50, 405),
 (56, 50),
 (56, 98),
 (56, 100),
 (56, 174),
 (56, 181),
 (69, 50),
 (79, 50),
 (79, 174),
 (98, 50),
 (98, 56),
 (98, 100),
 (98, 174),
 (98, 181),
 (100, 1),
 (100, 7),
 (100, 50),
 (100, 56),
 (100, 98),
 (100, 117),
 (100, 121),
 (100, 127),
 (100, 174),
 (100, 181),
 (100, 237),
 (117, 1),
 (117, 50),
 (117, 100),
 (117, 121),
 (117, 181),
 (121, 1),
 (121, 50),
 (121, 100),
 (121, 117),
 (121, 181),
 (121, 405),
 (127, 50),
 (127, 100),
 (127, 181),
 (172, 50),
 (172, 174),
 (172, 181),
 (173, 50),
 (174, 50),
 (174, 56),
 (174, 79),
 (174, 98),
 (174, 100),
 (174, 172),
 (174, 181),
 (174, 204),
 (174, 210),
 (181, 1),
 (181, 7),
 (181, 50),
